In [1]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output

# Load the CSV file (replace 'data.csv' with the path to your data)
data = pd.read_csv("data.csv")

# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'], format='%b %d %Y')

# List of apps
apps = ['instagram', 'phone', 'whatsapp', 'chatgpt', 'youtube', 'google', 'snapchat']
alice_cols = [f"{app}(Alice)" for app in apps]
bob_cols = [f"{app}(Bob)" for app in apps]

# Create Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Interactive App Usage Dashboard", style={'text-align': 'center'}),

    # Dropdown to select user(s)
    dcc.Dropdown(
        id='user-dropdown',
        options=[
            {'label': 'Alice', 'value': 'Alice'},
            {'label': 'Bob', 'value': 'Bob'},
            {'label': 'Both', 'value': 'Both'}
        ],
        value='Both',  # Default value
        style={'width': '50%', 'margin': '0 auto'}
    ),

    # Date Picker to select a specific day
    dcc.DatePickerSingle(
        id='date-picker',
        min_date_allowed=data['date'].min(),
        max_date_allowed=data['date'].max(),
        initial_visible_month=data['date'].max(),
        date=data['date'].max().date(),  # Default to the latest date in the dataset
        style={'display': 'block', 'margin': '20px auto'}
    ),

    # Button to trigger plot updates
    html.Button('Update Visualization', id='update-button', n_clicks=0, style={'display': 'block', 'margin': '20px auto'}),

    # Graphs
    dcc.Graph(id='usage-bar-graph'),
    dcc.Graph(id='usage-line-graph'),
    dcc.Graph(id='usage-pie-chart'),
    dcc.Graph(id='usage-scatter-plot'),
])

# Callback to update graphs based on user input
@app.callback(
    [Output('usage-bar-graph', 'figure'),
     Output('usage-line-graph', 'figure'),
     Output('usage-pie-chart', 'figure'),
     Output('usage-scatter-plot', 'figure')],
    [Input('user-dropdown', 'value'),
     Input('date-picker', 'date'),
     Input('update-button', 'n_clicks')]
)
def update_graph(user_selection, selected_date, n_clicks):
    # Filter data based on selected date
    selected_date = pd.to_datetime(selected_date)
    daily_data = data[data['date'] == selected_date]

    # Check if data exists for the selected date
    if daily_data.empty:
        return {}, {}, {}, {}

    if n_clicks > 0:
        if user_selection == 'Alice':
            # Bar Chart: Alice's Total App Usage
            alice_total = daily_data[alice_cols].sum()
            figure1 = go.Figure(data=[go.Bar(
                x=apps,
                y=alice_total,
                name="Alice",
                marker=dict(color='blue')
            )])
            figure1.update_layout(
                title="Alice's Total App Usage",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                barmode='group'
            )

            # Line Plot: Alice's App Usage on Selected Day
            figure2 = go.Figure(data=[go.Scatter(
                x=apps,
                y=alice_total,
                mode='lines+markers',
                name="Alice",
                marker=dict(color='blue')
            )])
            figure2.update_layout(
                title=f"Alice's App Usage on {selected_date.date()}",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                legend_title="Apps"
            )

            # Pie Chart: Alice's App Usage Distribution (on the selected date)
            figure3 = go.Figure(data=[go.Pie(labels=apps, values=alice_total, name="Alice", hole=0.3)])
            figure3.update_layout(title="Alice's App Usage Distribution")

            # Scatter Plot: Alice's vs Bob's App Usage (per-app comparison)
            bob_total = daily_data[bob_cols].sum()

            # Prepare scatter plot data for Alice vs Bob for each app
            figure4 = go.Figure()
            for app in apps:
                alice_usage = daily_data[f"{app}(Alice)"].values[0]  # Get single value for selected day
                bob_usage = daily_data[f"{app}(Bob)"].values[0]  # Get single value for selected day

                # Add a scatter trace for each app
                figure4.add_trace(go.Scatter(
                    x=[alice_usage],  # Alice's usage
                    y=[bob_usage],  # Bob's usage
                    mode='markers',
                    name=app,  # Label by app name
                    marker=dict(size=10, color='purple'),
                    text=[f"{app}: Alice = {alice_usage}, Bob = {bob_usage}"],  # Hover text
                    hoverinfo='text'  # Display hover text when the user hovers over each point
                ))

            figure4.update_layout(
                title="Comparison of Alice's and Bob's App Usage (Per-App)",
                xaxis_title="Alice's Usage (minutes)",
                yaxis_title="Bob's Usage (minutes)",
                legend_title="Apps"
            )

        elif user_selection == 'Bob':
            # Bar Chart: Bob's Total App Usage
            bob_total = daily_data[bob_cols].sum()
            figure1 = go.Figure(data=[go.Bar(
                x=apps,
                y=bob_total,
                name="Bob",
                marker=dict(color='red')
            )])
            figure1.update_layout(
                title="Bob's Total App Usage",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                barmode='group'
            )

            # Line Plot: Bob's App Usage on Selected Day
            figure2 = go.Figure(data=[go.Scatter(
                x=apps,
                y=bob_total,
                mode='lines+markers',
                name="Bob",
                marker=dict(color='red')
            )])
            figure2.update_layout(
                title=f"Bob's App Usage on {selected_date.date()}",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                legend_title="Apps"
            )

            # Pie Chart: Bob's App Usage Distribution (on the selected date)
            figure3 = go.Figure(data=[go.Pie(labels=apps, values=bob_total, name="Bob", hole=0.3)])
            figure3.update_layout(title="Bob's App Usage Distribution")

            # Scatter Plot: Alice's vs Bob's App Usage (per-app comparison)
            alice_total = daily_data[alice_cols].sum()

            # Prepare scatter plot data for Alice vs Bob for each app
            figure4 = go.Figure()
            for app in apps:
                alice_usage = daily_data[f"{app}(Alice)"].values[0]  # Get single value for selected day
                bob_usage = daily_data[f"{app}(Bob)"].values[0]  # Get single value for selected day

                # Add a scatter trace for each app
                figure4.add_trace(go.Scatter(
                    x=[alice_usage],  # Alice's usage
                    y=[bob_usage],  # Bob's usage
                    mode='markers',
                    name=app,  # Label by app name
                    marker=dict(size=10, color='purple'),
                    text=[f"{app}: Alice = {alice_usage}, Bob = {bob_usage}"],  # Hover text
                    hoverinfo='text'  # Display hover text when the user hovers over each point
                ))

            figure4.update_layout(
                title="Comparison of Alice's and Bob's App Usage (Per-App)",
                xaxis_title="Alice's Usage (minutes)",
                yaxis_title="Bob's Usage (minutes)",
                legend_title="Apps"
            )

        else:  # For 'Both' users
            # Bar Chart: Comparison of Alice and Bob's Total App Usage
            alice_total = daily_data[alice_cols].sum()
            bob_total = daily_data[bob_cols].sum()

            figure1 = go.Figure(data=[
                go.Bar(x=apps, y=alice_total, name="Alice", marker=dict(color='blue')),
                go.Bar(x=apps, y=bob_total, name="Bob", marker=dict(color='red'))
            ])
            figure1.update_layout(
                title="Comparison of App Usage Between Alice and Bob",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                barmode='group',  # Group bars for each app
                legend_title="Users"
            )

            # Line Plot: Comparison of Alice and Bob's App Usage on Selected Day
            figure2 = go.Figure()
            figure2.add_trace(go.Scatter(
                x=apps,
                y=alice_total,
                mode='lines+markers',
                name="Alice",
                marker=dict(color='blue')
            ))
            figure2.add_trace(go.Scatter(
                x=apps,
                y=bob_total,
                mode='lines+markers',
                name="Bob",
                marker=dict(color='red')
            ))
            figure2.update_layout(
                title=f"Alice and Bob's App Usage on {selected_date.date()}",
                xaxis_title="App",
                yaxis_title="Usage (minutes)",
                legend_title="Apps"
            )

            # Pie Chart: Combined App Usage Distribution for Alice and Bob
            combined_total = alice_total + bob_total
            figure3 = go.Figure(data=[go.Pie(labels=apps, values=combined_total, name="Combined Usage", hole=0.3)])
            figure3.update_layout(title="Combined App Usage Distribution (Alice + Bob)")

            # Scatter Plot: Alice's vs Bob's App Usage (per-app comparison)
            figure4 = go.Figure()
            for app in apps:
                alice_usage = daily_data[f"{app}(Alice)"].values[0]  # Get single value for selected day
                bob_usage = daily_data[f"{app}(Bob)"].values[0]  # Get single value for selected day

                figure4.add_trace(go.Scatter(
                    x=[alice_usage],
                    y=[bob_usage],
                    mode='markers',
                    name=app,
                    marker=dict(size=10, color='purple'),
                    text=[f"{app}: Alice = {alice_usage}, Bob = {bob_usage}"],
                    hoverinfo='text'
                ))

            figure4.update_layout(
                title="Comparison of Alice's and Bob's App Usage (Per-App)",
                xaxis_title="Alice's Usage (minutes)",
                yaxis_title="Bob's Usage (minutes)",
                legend_title="Apps"
            )

        return figure1, figure2, figure3, figure4  # Return all the graphs

    return {}, {}, {}, {}

if __name__ == '__main__':
    app.run_server(debug=True)
